# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("cities.csv")
weather_data

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity
humidity = weather_data["Humidity"]

max_humidity = np.max(humidity).astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius = 1)

# Add layer
fig.add_layer(heat_layer)



In [5]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrow_city_df = weather_data.loc[(weather_data["Max Temp"] > 70) & (weather_data["Max Temp"] < 80) & (weather_data["Cloudiness"] == 0), :]
narrow_city_df = narrow_city_df.dropna(how='any')
narrow_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
391,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = []
counter = 0

for index, row in narrow_city_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
  
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    jsn = response.json()
    hotelname = jsn['results'][0]['name']
    time.sleep(2)
 

  
    
    try:
        hotel_df.append(jsn['results'][0]['name'])
        print(f"Closest hotel to {lat} , {lng} is {hotelname}.")
    except:
        hotel_df.append("")
        print("Missing field/result... skipping.")
        
    print("------------")

    






Closest hotel to -13.71 , -76.2 is Pisco.
------------
Closest hotel to -20.63 , -46.0 is Capitólio.
------------
Closest hotel to -30.17 , -50.22 is CIDREIRA.
------------
Closest hotel to -20.56 , 164.28 is Koumac.
------------
Closest hotel to -20.3 , 44.28 is Morondava.
------------
Closest hotel to 15.34 , 74.49 is Jagalbet.
------------
Closest hotel to 35.17 , -2.93 is Nador.
------------
Closest hotel to 22.92 , 96.51 is Mogok.
------------
Closest hotel to -15.19 , 12.15 is Namibe.
------------
Closest hotel to 32.86 , 59.22 is Birjand.
------------


In [10]:
narrow_city_df["Hotel Name"] = hotel_df

In [11]:
narrow_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,Pisco
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Capitólio
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,CIDREIRA
114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,Koumac
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Morondava
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Jagalbet
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Nador
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Mogok
391,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40,Namibe
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,Birjand


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrow_city_df.iterrows()]
locations = narrow_city_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))